# HW 4|Problem 3

Use the linalg package, LU decompositin, and GaussSeidel method to solve 2y+z=-8, x-2y-3z=0, -x+y+2z=3.

In [98]:
#First, record the 2 matrices that matter

import numpy as np

equation = np.array([[1,-2,-3],[0,2,1],[-1,1,2]])
answer = np.array([0,-8,3])
print(equation, answer)

[[ 1 -2 -3]
 [ 0  2  1]
 [-1  1  2]] [ 0 -8  3]


In [99]:
#solve system above using linalg package

import time

start_time = time.time()
solution = np.linalg.solve(equation,answer)
t = time.time() - start_time
print('x =',solution)
print('Time ', t)

x = [-4. -5.  2.]
Time  0.0


In [100]:
#solve system using LU decomposition: I've ripped the LU code from the Lecture

import numpy as np
#============================================================================
def LUFactor(a, ipivot, n):
# Copy right, code from Titus textbook. 
#----------------------------------------------------------------------------
#  Performs LU factorization of (n x n) matrix a (diag(L) = 1). On exit,
#  replaces upper triangle and diagonal with U, and lower triangle, with L.
#  Uses partial pivoting on columns.
#  a      - coefficient matrix (n x n); LU decomposition on exit
#  ipivot - array of pivot row indexes (output)
#  det    - determinant of coefficient matrix (output).
#----------------------------------------------------------------------------
   det = 1e0
   for j in range(n):                                 # loop over columns
      for i in range(j):                             # elements of matrix U
         sum = a[i][j]
         for k in range(i): sum -= a[i][k]*a[k][j]
         a[i][j] = sum

      amax = 0e0
      for i in range(j,n):                           # elements of matrix L
         sum = a[i][j]                                 # undivided by pivot
         for k in range(j): sum -= a[i][k]*a[k][j]
         a[i][j] = sum
                                                            # determine pivot
         if (amax < np.fabs(a[i][j])): amax = np.fabs(a[i][j]); imax = i

      if (amax == 0e0): print("LUFactor: singular matrix !"); return 0e0

      ipivot[j] = imax                                # store pivot row index
                                                # interchange rows imax and j
      if (imax != j):                           # to put pivot on diagonal
         det = -det
         for k in range(n):
            t = a[imax][k]; a[imax][k] = a[j][k]; a[j][k] = t

      det *= a[j][j]                        # multiply determinant with pivot

      t = 1e0/a[j][j]                         # divide elements of L by pivot
      for i in range(j+1,n): a[i][j] *= t

   return det

#============================================================================
def LUSystem(a, ipivot, b, n):
# Copy right, code from Titus textbook. 
#----------------------------------------------------------------------------
#  Solves linear system a x = b of order n by LU factorization.
#  a      - LU decomposition of coefficient matrix (returned by LUFactor)
#  ipivot - array of pivot row indexes (input)
#  b      - vector of constant terms (input); solution x (on exit)
#----------------------------------------------------------------------------
   for i in range(n):                                     # solves Ly = b
      sum = b[int(ipivot[i])]
      b[int(ipivot[i])] = b[i]
      for j in range(i): sum -= a[i][j]*b[j]
      b[i] = sum

   for i in range(n-1,-1,-1):                                    # solves Ux = y
      sum = b[i]
      for j in range(i+1,n): sum -= a[i][j]*b[j]
      b[i] = sum/a[i][i]

equation = np.array([[0.,2.,1.],[1.,-2.,-3.],[-1.,1.,2.]])
answer = np.array([-8.,0.,3.])
#print(equation, answer)    
    
equation_copy=np.copy(equation)    #deep copy of an array
answer_copy=np.copy(answer)    #deep copy of an array
ipivot=np.zeros(3)

start_time = time.time()

de=LUFactor(equation,ipivot,3)
LUSystem(equation,ipivot,answer,3)

t = time.time() - start_time
print('x =',answer)
print('Time ', t)
#I learned the hard way that if you input integers the final answer is wrong. Must use float

x = [-4. -5.  2.]
Time  0.0


In [101]:
#solve system using GaussSeidel method.

#============================================================================
def GaussSeidel(a, b, x, n, init):
# Copy right, code from Titus textbook. 
#----------------------------------------------------------------------------
#  Solves linear system a x = b by the Gauss-Seidel method.
#  Convergence is ensure by left-multiplying the system with a^T.
#  a    - coefficient matrix (n x n)
#  b    - vector of constant terms
#  x    - initial approximation of solution (input); solution (output)
#  n    - order of system
#  err  - maximum relative error of the solution components
#  init - initialization option: 0 - refines initial approximation 
#                                1 - initializes solution
#----------------------------------------------------------------------------
   eps = 1e-15                                 # relative precision criterion
   itmax = 10000                                    # max no. of iterations

   s = [[0]*(n) for i in range(n)]           # matrices of reduced system
   t = [0]*(n)

   for i in range(n):                         # matrices of normal system
      for j in range(i+1):                      # by multiplication with aT
         s[i][j] = 0e0                            # store result in s and t
         for k in range(n): s[i][j] += a[k][i]*a[k][j]
         s[j][i] = s[i][j]

      t[i] = 0e0
      for j in range(n): t[i] += a[j][i]*b[j]

   for i in range(n):                # matrices s and t of reduced system
      f = -1e0/s[i][i]; t[i] /= s[i][i]
      for j in range(n): s[i][j] *= f

   if (init):
      for i in range(n): x[i] = t[i]                # initialize solution

   for k in range(itmax):                            # loop of iterations
      err = 0e0
      for i in range(n):
         delta = t[i]                                            # correction
         for j in range(n): delta += s[i][j]*x[j]
         x[i] += delta                        # new approximation to solution
         if (x[i]): delta /= x[i]                            # relative error
         if (np.fabs(delta) > err): err = np.fabs(delta)            # maximum error
       #  print('delta,err',delta,err)
            
      if (err <= eps): break                              # check convergence

   if (k > itmax-2): print("GaussSeidel: max. no. of iterations exceeded !")

   return err

equation = np.array([[0.,2.,1.],[1.,-2.,-3.],[-1.,1.,2.]])
answer = np.array([-8.,0.,3.])
solution = np.array([1.,1.,1.])

time_start = time.time()
GaussSeidel(equation, answer, solution, 3, 1)
t = time.time() - time_start
print('x=',solution)
print('Time', t)

x= [-4. -5.  2.]
Time 0.032911062240600586
